# Ejemplo Kafka

1. Inicializamos la SparkSession.

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split
import os
import pyspark
"""
scala_version = '2.12'  # TODO: Ensure this is correct
spark_version = pyspark.__version__

packages = [
    f'org.apache.spark:spark-sql-kafka-0-10_{scala_version}:{spark_version}',
    'org.apache.kafka:kafka-clients:3.5.1'
]

args = os.environ.get('PYSPARK_SUBMIT_ARGS', '')
if not args:
    args = f'--packages {",".join(packages)}'
    print('Using packages', packages)
    os.environ['PYSPARK_SUBMIT_ARGS'] = f'{args} pyspark-shell'
else:
    print(f'Found existing args: {args}') 
"""
spark = SparkSession.builder\
  .appName("kafka-example-1")\
  .config("spark.sql.legacy.timeParserPolicy", "LEGACY") \
  .getOrCreate()

sc =spark.sparkContext


:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/hadoop/.ivy2/cache
The jars for the packages stored in: /home/hadoop/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.kafka#kafka-clients added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-92ce04ee-447a-4364-bfd9-0c057f21797d;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.1.0 in central
	found io.delta#delta-storage;3.1.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.7 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found org.xerial.snappy#snappy-java;1.1.10.5 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.fi

2. Creamos el streaming DataFrame a partir de un flujo de entrada de Kafka:

In [2]:
# Subscribe to 1 topic
df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "kafka:9092") \
  .option("subscribe", "entrada") \
  .load()
df_str = df.selectExpr("CAST(value AS STRING) as value")
df_str

DataFrame[value: string]

3. Preparamos el DataFrame de salida:

In [3]:
from pyspark.sql.functions import col
words = df_str.select(explode(split(col("value"), " ")).alias("word"))
word_counts = words.groupBy("word").count()
output = word_counts.selectExpr(
    "CAST(word AS STRING) AS key",
    "CAST(count AS STRING) AS value"
)
output

DataFrame[key: string, value: string]

4. Iniciamos la consulta en streaming indicando el destino de los datos:

In [4]:
query = output.writeStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka:9092") \
    .option("topic", "salida") \
    .option("checkpointLocation", "file:///home/hadoop/kafka-wordcount-checkpoint11") \
    .outputMode("update") \
    .start()


[Stage 1:=============>                                          (49 + 2) / 200]

In [ ]:
!ls -lah /opt/spark/jars-extra | egrep -i "delta|spark"